## Imports

In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix
import time

## Preparing Matrices

In [2]:
# the number of features
n = 61188
#number of classes
K = 20
#training instances
M = 12000

training_labels = np.load('../res/lr_labels.npy')
training_data_sparse = sparse.load_npz('../res/lr_training_data.npz')
XT = training_data_sparse.transpose()
weights_matrix = np.random.rand(K, n + 1)
# weights_sparse is the W matrix in the pdf
weights_sparse = sparse.csr_matrix(weights_matrix)
print('weights shape:', weights_sparse.shape)
print('training data shape:', training_data_sparse.shape)
print('training data transpose shape:', XT.shape)


delta = lil_matrix(np.zeros((K, M)), dtype = np.int16)
for i, label in enumerate(training_labels):
    delta[label - 1, i] = 1
delta = delta.tocsr()
print('delta shape:', delta.shape)
# you see the index 1 mapping here: we need to be careful!
#how to get the true class of a training examples
print(delta[:,0].nonzero()[0][0] + 1)
print(training_labels[0])
print(sparse.issparse(delta))

weights shape: (20, 61189)
training data shape: (12000, 61189)
training data transpose shape: (61189, 12000)
delta shape: (20, 12000)
14
14
True


In [3]:
ETHA = 0.005
LAMBDA = 0.01
EPOCH = 3000

t0 = time.time()
print("Started at:", t0)
for i in range(EPOCH):
    # print(weights_sparse)    
    pxy = (weights_sparse * XT).todense()
    for j in range(M):
        pxy[:, j] /= np.sum(pxy[:, j])
    pxy = np.exp(pxy)
    for j in range(M):
        pxy[:, j] /= np.sum(pxy[:, j])
    # print(pxy)
    diff_exp = csr_matrix(delta - pxy)
    new_w = weights_sparse + ETHA * (diff_exp * training_data_sparse - weights_sparse.multiply(LAMBDA))
    weights_sparse = new_w
    # np.nan_to_num(weights_sparse, copy=False)

t1 = time.time()
print('Finished at:', t1, "\nTotal runtime:", t1 - t0)

Started at: 1584503386.5443327
Finished at: 1584506940.1326098 
Total runtime: 3553.5882771015167


In [ ]:
print(weights_sparse.shape)
print(weights_sparse)

## Testing

In [27]:
# xxx importing the normalized xs here BAD PROGRAMMING PRACTICE FIX WHEN NOT BLEEDING!
def test_weights(predictions, is_training=False):
    correct = 0
    for j in range(predictions.shape[0]):
        predictions[:,j] /= np.sum(predictions[:,j])
    predictions = np.exp(predictions)
    for j in range(predictions.shape[0]):
        predictions[:,j] /= np.sum(predictions[:,j])
        
    if is_training:
        for i in range(predictions.shape[1]):
            predicted = np.argmax(predictions[:,i]) + 1
            actual = training_labels[i]
            if predicted == actual:
                correct += 1
        print(f"Correct is {correct} and accuracy is {correct/predictions.shape[1]}")
    else:
        print('writing to file...')
        with open('../results/lr_out.csv', 'w') as out_stream:
            out_stream.write("id,class\n")
            ids = range(12001, 18775)
            for i in range(predictions.shape[1]):
                predicted_label = np.argmax(predictions[:,i]) + 1
                doc_id = ids[i]
                out_stream.write(f"{doc_id},{predicted_label}\n")
    print("Finished")

### training accuracy

In [ ]:
training_predictions = (weights_sparse * XT).todense()
test_weights(training_predictions, is_training=True)

### training accuracy

In [28]:
testing_data_sparse = sparse.load_npz('../res/lr_testing_data.npz')
test_XT = testing_data_sparse.transpose()
print('weights shape:', weights_sparse.shape)
print('testing data shape:', testing_data_sparse.shape)
print('testing data first column:', testing_data_sparse[:,0])
print('training data first column:', training_data_sparse[:,0])
# print('testing data transpose shape:', test_XT.shape)
# testing_predictions = (weights_sparse * test_XT).todense()
# test_weights(testing_predictions, is_training=False)

weights shape: (20, 61189)
testing data shape: (6774, 61189)
testing data first column:   (8, 0)	1
  (27, 0)	4
  (152, 0)	3
  (247, 0)	2
  (282, 0)	7
  (312, 0)	1
  (332, 0)	1
  (347, 0)	1
  (363, 0)	1
  (440, 0)	1
  (627, 0)	1
  (803, 0)	1
  (848, 0)	14
  (927, 0)	1
  (962, 0)	1
  (1019, 0)	11
  (1036, 0)	1
  (1115, 0)	5
  (1160, 0)	1
  (1227, 0)	1
  (1242, 0)	1
  (1302, 0)	1
  (1312, 0)	2
  (1543, 0)	1
  (1587, 0)	1
  :	:
  (4497, 0)	5
  (4624, 0)	3
  (4694, 0)	1
  (4706, 0)	1
  (4747, 0)	1
  (4858, 0)	1
  (4875, 0)	1
  (5064, 0)	1
  (5109, 0)	2
  (5280, 0)	1
  (5308, 0)	1
  (5323, 0)	1
  (5335, 0)	17
  (5569, 0)	3
  (5686, 0)	1
  (5765, 0)	1
  (5797, 0)	4
  (6054, 0)	1
  (6088, 0)	16
  (6098, 0)	1
  (6122, 0)	1
  (6234, 0)	1
  (6394, 0)	12
  (6555, 0)	2
  (6722, 0)	1
training data first column:   (288, 0)	4
  (312, 0)	1
  (410, 0)	1
  (703, 0)	1
  (722, 0)	25
  (905, 0)	4
  (917, 0)	1
  (1051, 0)	2
  (1553, 0)	1
  (1690, 0)	5
  (1699, 0)	7
  (1761, 0)	1
  (1972, 0)	1
  (2012, 0)	2
 